In [3]:
import pandas as pd
import numpy as np
import os

In [20]:
pd.set_option('display.max_columns', None)

## Join all the indicators

In [27]:
# Read all the 6 indicators (CSVs)
df_economic = pd.read_csv("../data/resources/readiness/melted_economic.csv")

df_governance = pd.read_csv("../data/resources/readiness/melted_governance.csv")
del df_governance["Name"]
df_social = pd.read_csv("../data/resources/readiness/melted_social.csv")
del df_social["Name"]

df_ecosystems = pd.read_csv("../data/resources/vulnerability/melted_ecosystems.csv")
del df_ecosystems["Name"]
df_habitat = pd.read_csv("../data/resources/vulnerability/melted_habitat.csv")
del df_habitat["Name"]
df_infrastructure = pd.read_csv("../data/resources/vulnerability/melted_infrastructure.csv")
del df_infrastructure["Name"]

# Also read political stability csv
df_politicalStability = pd.read_csv("../data/resources/indicators/id_gove_01/melted_governance.csv")
del df_politicalStability["Name"]

In [28]:
df_indicators = pd.merge(df_economic, df_governance, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_social, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_ecosystems, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_habitat, how="outer", on=["ISO3", "Year"])
df_indicators = pd.merge(df_indicators, df_infrastructure, how="outer", on=["ISO3", "Year"])

In [29]:
df_indicators = pd.merge(df_indicators, df_politicalStability, how="outer", on=["ISO3", "Year"])
df_indicators

,ISO3,Name,Year,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated
0,AFG,Afghanistan,1995,0.496497,False,0.138771,False,0.295982,False,0.516028,False,0.603153,False,0.383710,True,0.154928,False
1,ALB,Albania,1995,0.393305,False,0.383518,False,0.225699,False,0.484339,False,0.509650,False,0.474276,False,0.512667,False
2,DZA,Algeria,1995,0.416625,False,0.292892,False,0.194779,False,0.411816,False,0.467930,False,0.157379,False,0.263959,False
3,AND,Andorra,1995,0.410408,True,0.501894,True,0.156764,False,0.477737,True,0.527740,True,0.383710,True,0.578457,True
4,AGO,Angola,1995,0.289057,False,0.201718,False,0.083156,False,0.544357,False,0.648126,False,0.326570,False,0.216279,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,VEN,"Venezuela, Bolivarian Republic o",2020,0.111244,False,0.127607,False,0.331404,False,0.351461,False,0.604549,False,0.208577,False,0.309044,False
4988,VNM,Viet Nam,2020,0.490740,False,0.468500,False,0.316170,False,0.555078,False,0.433713,False,0.562195,False,0.557445,False
4989,YEM,Yemen,2020,0.379586,False,0.120174,False,0.243894,False,0.570795,False,0.611543,False,0.340751,False,0.110621,False
4990,ZMB,Zambia,2020,0.413829,False,0.399141,False,0.150245,False,0.433906,False,0.594931,False,0.560437,False,0.548028,False


In [30]:
# To armonize data, let's change "United states" as "USA"
invalid_rows = df_indicators["Name"] == "United States"
df_indicators.loc[invalid_rows, "Name"] = "USA"

In [31]:
# Create geometric mean for these 6 indicators (EGSEHI)
df_indicators["EGSEHI"] = df_indicators["value_economic"] * df_indicators["value_governance"] * df_indicators["value_social"] * \
                            df_indicators["value_ecosystems"] * df_indicators["value_habitat"] * df_indicators["value_infrastructure"]

root = 6
df_indicators[f"EGSEHI_{root}root"] = np.power(df_indicators["EGSEHI"], root)

In [35]:
df_indicators["Country"] = df_indicators["Name"].str.upper()
del df_indicators["Name"]

In [36]:
df_indicators.to_csv("../data/indicators_conformed_file.csv", index=False)

In [37]:
df_indicators

,ISO3,Year,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root,Country
0,AFG,1995,0.496497,False,0.138771,False,0.295982,False,0.516028,False,0.603153,False,0.383710,True,0.154928,False,0.002435,2.086886e-16,AFGHANISTAN
1,ALB,1995,0.393305,False,0.383518,False,0.225699,False,0.484339,False,0.509650,False,0.474276,False,0.512667,False,0.003986,4.008480e-15,ALBANIA
2,DZA,1995,0.416625,False,0.292892,False,0.194779,False,0.411816,False,0.467930,False,0.157379,False,0.263959,False,0.000721,1.402677e-19,ALGERIA
3,AND,1995,0.410408,True,0.501894,True,0.156764,False,0.477737,True,0.527740,True,0.383710,True,0.578457,True,0.003124,9.292030e-16,ANDORRA
4,AGO,1995,0.289057,False,0.201718,False,0.083156,False,0.544357,False,0.648126,False,0.326570,False,0.216279,False,0.000559,3.039842e-20,ANGOLA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,VEN,2020,0.111244,False,0.127607,False,0.331404,False,0.351461,False,0.604549,False,0.208577,False,0.309044,False,0.000208,8.213285e-23,"VENEZUELA, BOLIVARIAN REPUBLIC O"
4988,VNM,2020,0.490740,False,0.468500,False,0.316170,False,0.555078,False,0.433713,False,0.562195,False,0.557445,False,0.009838,9.068718e-13,VIET NAM
4989,YEM,2020,0.379586,False,0.120174,False,0.243894,False,0.570795,False,0.611543,False,0.340751,False,0.110621,False,0.001323,5.370178e-18,YEMEN
4990,ZMB,2020,0.413829,False,0.399141,False,0.150245,False,0.433906,False,0.594931,False,0.560437,False,0.548028,False,0.003590,2.142007e-15,ZAMBIA


### ADD reporter code to countries in indicators_df

In [38]:
# final_production_df["Country"] = final_production_df["Country"].str.upper()
# final_production_df["Product"] = final_production_df["Product"].str.upper()

# Attach country code to df
countries_list = [{'reporterCode': 36, 'reporterDesc': 'Australia'},
 {'reporterCode': 40, 'reporterDesc': 'Austria'},
 {'reporterCode': 56, 'reporterDesc': 'Belgium'},
 {'reporterCode': 68, 'reporterDesc': 'Bolivia (Plurinational State of)'},
 {'reporterCode': 68, 'reporterDesc': 'Bolivia'},
 {'reporterCode': 76, 'reporterDesc': 'Brazil'},
 {'reporterCode': 156, 'reporterDesc': 'China'},
 {'reporterCode': 251, 'reporterDesc': 'France'},
 {'reporterCode': 276, 'reporterDesc': 'Germany'},
 {'reporterCode': 278, 'reporterDesc': 'GERMAN DEM. REP'},
 {'reporterCode': 320, 'reporterDesc': 'Guatemala'},
 {'reporterCode': 360, 'reporterDesc': 'Indonesia'},
 {'reporterCode': 380, 'reporterDesc': 'Italy'},
 {'reporterCode': 417, 'reporterDesc': 'Kyrgyzstan'},
 {'reporterCode': 504, 'reporterDesc': 'Morocco'},
 {'reporterCode': 528, 'reporterDesc': 'Netherlands'},
 {'reporterCode': 710, 'reporterDesc': 'South Africa'},
 {'reporterCode': 757, 'reporterDesc': 'Switzerland'},
 {'reporterCode': 764, 'reporterDesc': 'Thailand'},
 {'reporterCode': 792, 'reporterDesc': 'Türkiye'},
 {'reporterCode': 807, 'reporterDesc': 'North Macedonia'},
 {'reporterCode': 842, 'reporterDesc': 'USA'},
 {'reporterCode': 842, 'reporterDesc': 'UNITED STATES'},
 
 {'reporterCode': 32, 'reporterDesc': 'Argentina'},
 {'reporterCode': 100, 'reporterDesc': 'Bulgaria'},
 {'reporterCode': 112, 'reporterDesc': 'Belarus'},
 {'reporterCode': 124, 'reporterDesc': 'Canada'},
 {'reporterCode': 144, 'reporterDesc': 'Sri Lanka'},
 {'reporterCode': 152, 'reporterDesc': 'Chile'},
 {'reporterCode': 170, 'reporterDesc': 'Colombia'},
 {'reporterCode': 196, 'reporterDesc': 'Cyprus'},
 {'reporterCode': 222, 'reporterDesc': 'El Salvador'},
 {'reporterCode': 233, 'reporterDesc': 'Estonia'},
 {'reporterCode': 300, 'reporterDesc': 'Greece'},
 {'reporterCode': 372, 'reporterDesc': 'Ireland'},
 {'reporterCode': 398, 'reporterDesc': 'Kazakhstan'},
 {'reporterCode': 404, 'reporterDesc': 'Kenya'},
 {'reporterCode': 410, 'reporterDesc': 'Rep. of Korea'},
 {'reporterCode': 410, 'reporterDesc': 'KOREA, SOUTH'},
 {'reporterCode': 440, 'reporterDesc': 'Lithuania'},
 {'reporterCode': 484, 'reporterDesc': 'Mexico'},
 {'reporterCode': 512, 'reporterDesc': 'Oman'},
 {'reporterCode': 516, 'reporterDesc': 'Namibia'},
 {'reporterCode': 604, 'reporterDesc': 'Peru'},
 {'reporterCode': 620, 'reporterDesc': 'Portugal'},
 {'reporterCode': 642, 'reporterDesc': 'Romania'},
 {'reporterCode': 643, 'reporterDesc': 'Russian Federation'},
 {'reporterCode': 643, 'reporterDesc': 'RUSSIA'}, 
 {'reporterCode': 682, 'reporterDesc': 'Saudi Arabia'},
 {'reporterCode': 686, 'reporterDesc': 'Senegal'},
 {'reporterCode': 699, 'reporterDesc': 'India'},
 {'reporterCode': 702, 'reporterDesc': 'Singapore'},
 {'reporterCode': 716, 'reporterDesc': 'Zimbabwe'},
 {'reporterCode': 724, 'reporterDesc': 'Spain'},
 {'reporterCode': 748, 'reporterDesc': 'Eswatini'},
 {'reporterCode': 752, 'reporterDesc': 'Sweden'},
 {'reporterCode': 780, 'reporterDesc': 'Trinidad and Tobago'},
 {'reporterCode': 784, 'reporterDesc': 'United Arab Emirates'},
 {'reporterCode': 788, 'reporterDesc': 'Tunisia'},
 {'reporterCode': 795, 'reporterDesc': 'Turkmenistan'},
 {'reporterCode': 804, 'reporterDesc': 'Ukraine'},
 {'reporterCode': 826, 'reporterDesc': 'United Kingdom'},
 {'reporterCode': 891, 'reporterDesc': 'Serbia and Montenegro (...2005)'},
 {'reporterCode': 31, 'reporterDesc': 'Azerbaijan'},
 {'reporterCode': 44, 'reporterDesc': 'Bahamas'},
 {'reporterCode': 120, 'reporterDesc': 'Cameroon'},
 {'reporterCode': 191, 'reporterDesc': 'Croatia'},
 {'reporterCode': 203, 'reporterDesc': 'Czechia'},
 {'reporterCode': 203, 'reporterDesc': 'CZECH REPUBLIC'},
 {'reporterCode': 208, 'reporterDesc': 'Denmark'},
 {'reporterCode': 246, 'reporterDesc': 'Finland'},
 {'reporterCode': 266, 'reporterDesc': 'Gabon'},
 {'reporterCode': 268, 'reporterDesc': 'Georgia'},
 {'reporterCode': 344, 'reporterDesc': 'China, Hong Kong SAR'},
 {'reporterCode': 344, 'reporterDesc': 'HONG KONG'},
 {'reporterCode': 364, 'reporterDesc': 'Iran'},
 {'reporterCode': 392, 'reporterDesc': 'Japan'},
 {'reporterCode': 422, 'reporterDesc': 'Lebanon'},
 {'reporterCode': 480, 'reporterDesc': 'Mauritius'},
 {'reporterCode': 490, 'reporterDesc': 'Other Asia, nes'},
 {'reporterCode': 579, 'reporterDesc': 'Norway'},
 {'reporterCode': 703, 'reporterDesc': 'Slovakia'},
 {'reporterCode': 704, 'reporterDesc': 'Viet Nam'},
 {'reporterCode': 704, 'reporterDesc': 'VIETNAM'},
 {'reporterCode': 705, 'reporterDesc': 'Slovenia'},
 {'reporterCode': 862, 'reporterDesc': 'Venezuela'},
 {'reporterCode': 192, 'reporterDesc': 'Cuba'},
 {'reporterCode': 450, 'reporterDesc': 'Madagascar'},
 {'reporterCode': 608, 'reporterDesc': 'Philippines'},
 {'reporterCode': 736, 'reporterDesc': 'Sudan (...2011)'},
 {'reporterCode': 8, 'reporterDesc': 'Albania'},
 {'reporterCode': 72, 'reporterDesc': 'Botswana'},
 {'reporterCode': 348, 'reporterDesc': 'Hungary'},
 {'reporterCode': 384, 'reporterDesc': "Côte d'Ivoire"},
 {'reporterCode': 384, 'reporterDesc': "COTE D'IVOIRE"},
 {'reporterCode': 400, 'reporterDesc': 'Jordan'},
 {'reporterCode': 428, 'reporterDesc': 'Latvia'},
 {'reporterCode': 442, 'reporterDesc': 'Luxembourg'},
 {'reporterCode': 458, 'reporterDesc': 'Malaysia'},
 {'reporterCode': 470, 'reporterDesc': 'Malta'},
 {'reporterCode': 496, 'reporterDesc': 'Mongolia'},
 {'reporterCode': 508, 'reporterDesc': 'Mozambique'},
 {'reporterCode': 554, 'reporterDesc': 'New Zealand'},
 {'reporterCode': 562, 'reporterDesc': 'Niger'},
 {'reporterCode': 616, 'reporterDesc': 'Poland'},
 {'reporterCode': 634, 'reporterDesc': 'Qatar'},
 {'reporterCode': 659, 'reporterDesc': 'Saint Kitts and Nevis'},
 {'reporterCode': 740, 'reporterDesc': 'Suriname'},
 {'reporterCode': 818, 'reporterDesc': 'Egypt'},
 {'reporterCode': 894, 'reporterDesc': 'Zambia'},
 {'reporterCode': 51, 'reporterDesc': 'Armenia'},
 {'reporterCode': 340, 'reporterDesc': 'Honduras'},
 {'reporterCode': 466, 'reporterDesc': 'Mali'},
 {'reporterCode': 598, 'reporterDesc': 'Papua New Guinea'},
 {'reporterCode': 20, 'reporterDesc': 'Andorra'},
 {'reporterCode': 108, 'reporterDesc': 'Burundi'},
 {'reporterCode': 116, 'reporterDesc': 'Cambodia'},
 {'reporterCode': 140, 'reporterDesc': 'Central African Rep.'},
 {'reporterCode': 140, 'reporterDesc': 'CENTRAL AFRICAN REPUBLIC'},
 {'reporterCode': 188, 'reporterDesc': 'Costa Rica'},
 {'reporterCode': 204, 'reporterDesc': 'Benin'},
 {'reporterCode': 218, 'reporterDesc': 'Ecuador'},
 {'reporterCode': 231, 'reporterDesc': 'Ethiopia'},
 {'reporterCode': 242, 'reporterDesc': 'Fiji'},
 {'reporterCode': 258, 'reporterDesc': 'French Polynesia'},
 {'reporterCode': 270, 'reporterDesc': 'Gambia'},
 {'reporterCode': 288, 'reporterDesc': 'Ghana'},
 {'reporterCode': 324, 'reporterDesc': 'Guinea'},
 {'reporterCode': 328, 'reporterDesc': 'Guyana'},
 {'reporterCode': 376, 'reporterDesc': 'Israel'},
 {'reporterCode': 414, 'reporterDesc': 'Kuwait'},
 {'reporterCode': 558, 'reporterDesc': 'Nicaragua'},
 {'reporterCode': 591, 'reporterDesc': 'Panama'},
 {'reporterCode': 762, 'reporterDesc': 'Tajikistan'},
 {'reporterCode': 834, 'reporterDesc': 'United Rep. of Tanzania'},
 {'reporterCode': 834, 'reporterDesc': 'TANZANIA'}, 
 {'reporterCode': 854, 'reporterDesc': 'Burkina Faso'},
 {'reporterCode': 858, 'reporterDesc': 'Uruguay'},
 {'reporterCode': 132, 'reporterDesc': 'Cabo Verde'},
 {'reporterCode': 132, 'reporterDesc': 'CAPE VERDE'},
 {'reporterCode': 48, 'reporterDesc': 'Bahrain'},
 {'reporterCode': 388, 'reporterDesc': 'Jamaica'},
 {'reporterCode': 478, 'reporterDesc': 'Mauritania'},
 {'reporterCode': 12, 'reporterDesc': 'Algeria'},
 {'reporterCode': 28, 'reporterDesc': 'Antigua and Barbuda'},
 {'reporterCode': 540, 'reporterDesc': 'New Caledonia'},
 {'reporterCode': 52, 'reporterDesc': 'Barbados'},
 {'reporterCode': 566, 'reporterDesc': 'Nigeria'},
 {'reporterCode': 800, 'reporterDesc': 'Uganda'},
 {'reporterCode': 426, 'reporterDesc': 'Lesotho'},
 {'reporterCode': 174, 'reporterDesc': 'Comoros'},
 {'reporterCode': 50, 'reporterDesc': 'Bangladesh'},
 {'reporterCode': 352, 'reporterDesc': 'Iceland'},
 {'reporterCode': 454, 'reporterDesc': 'Malawi'},


 {'reporterCode': 662, 'reporterDesc': 'Saint Lucia'},
 {'reporterCode': 214, 'reporterDesc': 'Dominican Rep.'},
 {'reporterCode': 214, 'reporterDesc': 'DOMINICAN REPUBLIC'},
 {'reporterCode': 882, 'reporterDesc': 'Samoa'},
 {'reporterCode': 646, 'reporterDesc': 'Rwanda'},
 {'reporterCode': 96, 'reporterDesc': 'Brunei Darussalam'},
 {'reporterCode': 96, 'reporterDesc': 'BRUNEI'},
 {'reporterCode': 776, 'reporterDesc': 'Tonga'},
 {'reporterCode': 600, 'reporterDesc': 'Paraguay'},
 {'reporterCode': 760, 'reporterDesc': 'Syria'},
 {'reporterCode': 498, 'reporterDesc': 'Rep. of Moldova'},
 {'reporterCode': 498, 'reporterDesc': 'MOLDOVA'},
 {'reporterCode': 768, 'reporterDesc': 'Togo'},
 {'reporterCode': 90, 'reporterDesc': 'Solomon Isds'},
 {'reporterCode': 90, 'reporterDesc': 'SOLOMON ISLANDS'},
 {'reporterCode': 690, 'reporterDesc': 'Seychelles'},
 {'reporterCode': 70, 'reporterDesc': 'Bosnia Herzegovina'},
 {'reporterCode': 70, 'reporterDesc': 'BOSNIA-HERZEGOVINA'},
 {'reporterCode': 500, 'reporterDesc': 'Montserrat'},
 {'reporterCode': 586, 'reporterDesc': 'Pakistan'},
 {'reporterCode': 524, 'reporterDesc': 'Nepal'},
 {'reporterCode': 887, 'reporterDesc': 'Yemen'},
 {'reporterCode': 887, 'reporterDesc': 'YEMEN ARAB REPUBLIC'}, # norte
 {'reporterCode': 887, 'reporterDesc': 'YEMEN, PDR'},
 {'reporterCode': 626, 'reporterDesc': 'Timor-Leste'},
 {'reporterCode': 798, 'reporterDesc': 'Tuvalu'},
 {'reporterCode': 64, 'reporterDesc': 'Bhutan'},
 {'reporterCode': 670, 'reporterDesc': 'Saint Vincent and the Grenadines'},
 {'reporterCode': 184, 'reporterDesc': 'Cook Isds'},
 {'reporterCode': 84, 'reporterDesc': 'Belize'},
 {'reporterCode': 530, 'reporterDesc': 'Netherlands Antilles (...2010)'},
 {'reporterCode': 688, 'reporterDesc': 'Serbia'},
 {'reporterCode': 548, 'reporterDesc': 'Vanuatu'},
 {'reporterCode': 275, 'reporterDesc': 'State of Palestine'},
 {'reporterCode': 434, 'reporterDesc': 'Libya'},
 {'reporterCode': 178, 'reporterDesc': 'Congo'},
 {'reporterCode': 178, 'reporterDesc': 'CONGO, REP.'},
 {'reporterCode': 24, 'reporterDesc': 'Angola'},
 {'reporterCode': 296, 'reporterDesc': 'Kiribati'},
 {'reporterCode': 499, 'reporterDesc': 'Montenegro'},
 {'reporterCode': 234, 'reporterDesc': 'Faeroe Isds'},
 {'reporterCode': 533, 'reporterDesc': 'Aruba'},
 {'reporterCode': 796, 'reporterDesc': 'Turks and Caicos Isds'},
 {'reporterCode': 104, 'reporterDesc': 'Myanmar'},
 {'reporterCode': 212, 'reporterDesc': 'Dominica'},
 {'reporterCode': 418, 'reporterDesc': "Lao People's Dem. Rep."},
 {'reporterCode': 304, 'reporterDesc': 'Greenland'},
 {'reporterCode': 368, 'reporterDesc': 'Iraq'},
 
 {'reporterCode': 729, 'reporterDesc': 'Sudan'},
 {'reporterCode': 446, 'reporterDesc': 'China, Macao SAR'},
 {'reporterCode': 308, 'reporterDesc': 'Grenada'},
 {'reporterCode': 585, 'reporterDesc': 'Palau'},
 {'reporterCode': 694, 'reporterDesc': 'Sierra Leone'},
 {'reporterCode': 60, 'reporterDesc': 'Bermuda'},
 {'reporterCode': 4, 'reporterDesc': 'Afghanistan'},
 {'reporterCode': 180, 'reporterDesc': 'Dem. Rep. of the Congo'},
 {'reporterCode': 180, 'reporterDesc': 'CONGO, D.R.'},
 {'reporterCode': 860, 'reporterDesc': 'Uzbekistan'},
 {'reporterCode': 136, 'reporterDesc': 'Cayman Isds'},

 {'reporterCode': 430, 'reporterDesc': 'Liberia'},
 {'reporterCode': 891, 'reporterDesc': 'Serbia and montenegro'},
 {'reporterCode': 332, 'reporterDesc': 'HAITI'},
 {'reporterCode': 792, 'reporterDesc': 'TURKEY'},
 {'reporterCode': 890, 'reporterDesc': 'FMR YUGOSLAVIA'},
 {'reporterCode': 890, 'reporterDesc': 'YUGOSLAVIA'},
 {'reporterCode': 200, 'reporterDesc': 'CZECHOSLOVAKIA'},
#  {'reporterCode': , 'reporterDesc': 'FMR RHODESIA NYAS'},
 {'reporterCode': 748, 'reporterDesc': 'SWAZILAND'},
 {'reporterCode': 408, 'reporterDesc': "DEM. PEOPLE'S REP. OF KOREA"},
 {'reporterCode': 408, 'reporterDesc': "KOREA, NORTH"},
 {'reporterCode': 807, 'reporterDesc': "TFYR OF MACEDONIA"},
#  {'reporterCode': , 'reporterDesc': "OTHER AFRICA, NES"},
#  {'reporterCode': , 'reporterDesc': 'OTHER EUROPE, NES'},
 {'reporterCode': 232, 'reporterDesc': "ERITREA"},
 {'reporterCode': 148, 'reporterDesc': "CHAD"},
 {'reporterCode': 226, 'reporterDesc': "EQUATORIAL GUINEA"},
 {'reporterCode': 254, 'reporterDesc': "FRENCH GUIANA"},
 {'reporterCode': 736, 'reporterDesc': "SOUTH SUDAN"},
 {'reporterCode': 706, 'reporterDesc': "SOMALIA"},
#  {'reporterCode': , 'reporterDesc': ""},
 ]
countries_list = [{"reporterCode": country["reporterCode"], "Country": country["reporterDesc"].upper()} for country in countries_list]
df_countries_code = pd.json_normalize(countries_list)

df_indicators2 = pd.merge(df_indicators, df_countries_code, how="left", left_on="Country", right_on="Country")
df_indicators2

,ISO3,Year,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root,Country,reporterCode
0,AFG,1995,0.496497,False,0.138771,False,0.295982,False,0.516028,False,0.603153,False,0.383710,True,0.154928,False,0.002435,2.086886e-16,AFGHANISTAN,4.0
1,ALB,1995,0.393305,False,0.383518,False,0.225699,False,0.484339,False,0.509650,False,0.474276,False,0.512667,False,0.003986,4.008480e-15,ALBANIA,8.0
2,DZA,1995,0.416625,False,0.292892,False,0.194779,False,0.411816,False,0.467930,False,0.157379,False,0.263959,False,0.000721,1.402677e-19,ALGERIA,12.0
3,AND,1995,0.410408,True,0.501894,True,0.156764,False,0.477737,True,0.527740,True,0.383710,True,0.578457,True,0.003124,9.292030e-16,ANDORRA,20.0
4,AGO,1995,0.289057,False,0.201718,False,0.083156,False,0.544357,False,0.648126,False,0.326570,False,0.216279,False,0.000559,3.039842e-20,ANGOLA,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,VEN,2020,0.111244,False,0.127607,False,0.331404,False,0.351461,False,0.604549,False,0.208577,False,0.309044,False,0.000208,8.213285e-23,"VENEZUELA, BOLIVARIAN REPUBLIC O",NaN
4988,VNM,2020,0.490740,False,0.468500,False,0.316170,False,0.555078,False,0.433713,False,0.562195,False,0.557445,False,0.009838,9.068718e-13,VIET NAM,704.0
4989,YEM,2020,0.379586,False,0.120174,False,0.243894,False,0.570795,False,0.611543,False,0.340751,False,0.110621,False,0.001323,5.370178e-18,YEMEN,887.0
4990,ZMB,2020,0.413829,False,0.399141,False,0.150245,False,0.433906,False,0.594931,False,0.560437,False,0.548028,False,0.003590,2.142007e-15,ZAMBIA,894.0


In [39]:
# DATA with no reporter code
df_indicators2[df_indicators2["reporterCode"].isna()]["Country"].unique()

array(['BOLIVIA, PLURINATIONAL STATE OF', 'BOSNIA AND HERZEGOVINA',
       'CONGO, THE DEMOCRATIC REPUBLIC O', 'DJIBOUTI', 'GUINEA-BISSAU',
       'IRAN, ISLAMIC REPUBLIC OF', "KOREA, DEMOCRATIC PEOPLE'S REPUB",
       'KOREA, REPUBLIC OF', "LAO PEOPLE'S DEMOCRATIC REPUBLIC",
       'LIBYAN ARAB JAMAHIRIYA', 'LIECHTENSTEIN', 'MACEDONIA', 'MALDIVES',
       'MARSHALL ISLANDS', 'MICRONESIA, FEDERATED STATES OF',
       'MOLDOVA, REPUBLIC OF', 'MONACO', 'NAURU', 'SAN MARINO',
       'SAO TOME AND PRINCIPE', 'SYRIAN ARAB REPUBLIC',
       'TANZANIA, UNITED REPUBLIC OF', 'VENEZUELA, BOLIVARIAN REPUBLIC O'],
      dtype=object)

### Production

In [24]:
df_production = pd.read_csv("../data/production/Production_one_file_2.csv")
df_production["Year"] = pd.to_numeric(df_production["Year"], downcast="integer")
df_production["Country"] = df_production["Country"].str.upper()
df_production["Product"] = df_production["Product"].str.upper()
df_production.rename(columns={"Value": "Domestic Production Value (P_AC)"}, inplace=True)

In [25]:
df_production

,Country,Product,cmdCode,Source,Year,Domestic Production Value (P_AC),Value (Ton),qtyUnitAbbr,reporterCode
0,ALBANIA,IRON ORE (FE-CONT.),2601.0,org,1984,4.395020e+08,439502.0,kg,8.0
1,ALGERIA,IRON ORE (FE-CONT.),2601.0,org,1984,2.000000e+09,2000000.0,kg,12.0
2,ARGENTINA,IRON ORE (FE-CONT.),2601.0,org,1984,3.455760e+08,345576.0,kg,32.0
3,AUSTRALIA,IRON ORE (FE-CONT.),2601.0,org,1984,5.688500e+10,56885000.0,kg,36.0
4,AUSTRIA,IRON ORE (FE-CONT.),2601.0,org,1984,1.137872e+09,1137872.0,kg,40.0
...,...,...,...,...,...,...,...,...,...
96380,SPAIN,URANIUM (U3O8-CONT.),261210.0,org,2020,NaN,NaN,kg,724.0
96381,UKRAINE,URANIUM (U3O8-CONT.),261210.0,org,2020,8.770000e+05,877.0,kg,804.0
96382,UNITED STATES,URANIUM (U3O8-CONT.),261210.0,org,2020,7.000000e+03,7.0,kg,842.0
96383,UZBEKISTAN,URANIUM (U3O8-CONT.),261210.0,org,2020,4.127000e+06,4127.0,kg,860.0


## Join Production and EGSEHI

In [16]:
df_production_egsehi = pd.merge(df_production, df_indicators, how="left", left_on=["Year", "Country"], right_on=["Year", "Name"])
del df_production_egsehi["Name"]

In [21]:
df_production_egsehi

,Country,Product,cmdCode,Source,Year,Domestic Production Value (P_AC),Value (Ton),qtyUnitAbbr,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root
0,ALBANIA,IRON ORE (FE-CONT.),2601.0,org,1984,4.395020e+08,439502.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALGERIA,IRON ORE (FE-CONT.),2601.0,org,1984,2.000000e+09,2000000.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARGENTINA,IRON ORE (FE-CONT.),2601.0,org,1984,3.455760e+08,345576.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AUSTRALIA,IRON ORE (FE-CONT.),2601.0,org,1984,5.688500e+10,56885000.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AUSTRIA,IRON ORE (FE-CONT.),2601.0,org,1984,1.137872e+09,1137872.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96380,SPAIN,URANIUM (U3O8-CONT.),261210.0,org,2020,NaN,NaN,kg,ESP,0.498795,False,0.646452,False,0.462754,False,0.281208,False,0.373402,False,0.335442,False,0.638047,False,0.005256,2.107548e-14
96381,UKRAINE,URANIUM (U3O8-CONT.),261210.0,org,2020,8.770000e+05,877.0,kg,UKR,0.421433,False,0.363488,False,0.496721,False,0.426281,False,0.469000,False,0.292860,False,0.369888,False,0.004455,7.819201e-15
96382,UNITED STATES,URANIUM (U3O8-CONT.),261210.0,org,2020,7.000000e+03,7.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96383,UZBEKISTAN,URANIUM (U3O8-CONT.),261210.0,org,2020,4.127000e+06,4127.0,kg,UZB,0.526698,False,0.328749,False,0.302127,False,0.506797,False,0.422226,False,0.267437,False,0.494995,False,0.002994,7.199459e-16


### Join HHI index table with production JOIN EGSEHI table

In [22]:
df_hhi_master = pd.read_csv("../data/hhi_data/df_hhi_master.csv")
df_hhi_master

c:\Users\zacat\OneDrive\Documentos\Investigacion\investigacion-gnn-cic\rb_venv3715\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (0,1,3,9,32,35,43,47,49,51,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Country,Product_x,cmdCode,Source,Year,Value,Value (Ton),qtyUnitAbbr,reporterCode,reporterDesc_x,Total_value_YearProduct,Share in % (production),Share HHI Production,HHI_production,HHI_production_scaled,typeCode,freqCode,refPeriodId,refYear,refMonth,reporterISO,reporterDesc_y,flowCode,flowDesc,partnerCode,partnerISO,partnerDesc,partner2Code,partner2ISO,partner2Desc,classificationCode,classificationSearchCode,isOriginalClassification,cmdDesc,aggrLevel,isLeaf,customsCode,customsDesc,mosCode,motCode,motDesc,qtyUnitCode,qty,isQtyEstimated,altQtyUnitCode,altQtyUnitAbbr,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Product_y,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports,HHI_exports_scaled
0,ALBANIA,IRON ORE (FE-CONT.),2601.0,org,2000,5.000000e+06,5000.0,kg,8.0,ALBANIA,6.058481e+11,0.000825,6.811025e-07,1202.379321,0.075656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALGERIA,IRON ORE (FE-CONT.),2601.0,org,2000,7.200000e+08,720000.0,kg,12.0,ALGERIA,6.058481e+11,0.118842,1.412334e-02,1202.379321,0.075656,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARGENTINA,IRON ORE (FE-CONT.),2601.0,org,2000,0.000000e+00,0.0,kg,32.0,ARGENTINA,6.058481e+11,0.000000,0.000000e+00,1202.379321,0.075656,C,A,20000101.0,2000.0,52.0,ARG,Argentina,X,Export,0.0,W00,World,0.0,W00,World,H1,HS,True,Iron ores and concentrates; including roasted ...,4.0,False,C00,TOTAL CPC,0.0,0.0,TOTAL MOT,8.0,6.811000e+04,False,-1.0,NaN,NaN,False,6.811000e+04,False,NaN,False,NaN,1.093500e+04,1.093500e+04,0.0,False,False,IRON,2.743603e+11,0.000025,6.162818e-10,3836.182057,0.342631
3,AUSTRALIA,IRON ORE (FE-CONT.),2601.0,org,2000,1.077287e+11,107728740.0,kg,36.0,AUSTRALIA,6.058481e+11,17.781476,3.161809e+02,1202.379321,0.075656,C,A,20000101.0,2000.0,52.0,AUS,Australia,X,Export,0.0,W00,World,0.0,W00,World,H1,HS,True,Iron ores and concentrates; including roasted ...,4.0,False,C00,TOTAL CPC,0.0,0.0,TOTAL MOT,8.0,1.652031e+11,False,-1.0,NaN,NaN,False,1.652031e+11,False,NaN,False,NaN,2.560404e+09,2.560404e+09,0.0,False,False,IRON,2.743603e+11,60.213923,3.625716e+03,3836.182057,0.342631
4,AUSTRIA,IRON ORE (FE-CONT.),2601.0,org,2000,5.950240e+08,595024.0,kg,40.0,AUSTRIA,6.058481e+11,0.098213,9.645870e-03,1202.379321,0.075656,C,A,20000101.0,2000.0,52.0,AUT,Austria,X,Export,0.0,W00,World,0.0,W00,World,H1,HS,True,Iron ores and concentrates; including roasted ...,4.0,False,C00,TOTAL CPC,0.0,0.0,TOTAL MOT,8.0,2.800000e+03,False,-1.0,NaN,NaN,False,2.800000e+03,False,NaN,False,NaN,1.340000e+02,1.340000e+02,0.0,False,False,IRON,2.743603e+11,0.000001,1.041534e-12,3836.182057,0.342631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71288,NaN,NaN,261510.0,NaN,2020,NaN,NaN,kg,752.0,NaN,NaN,NaN,NaN,NaN,NaN,C,A,20200101.0,2020.0,52.0,SWE,Sweden,X,Export,0.0,W00,World,0.0,W00,World,H5,HS,True,Zirconium ores and concentrates,6.0,True,C00,TOTAL CPC,0.0,0.0,TOTAL MOT,8.0,4.570543e+04,True,-1.0,NaN,0.0,False,4.570543e+04,True,0.0,False,0.0,2.909783e+04,2.909783e+04,6.0,False,True,ZIRCONIUM,5.196047e+08,0.008796,7.737300e-05,1210.445136,0.052793
71289,NaN,NaN,261510.0,NaN,2020,NaN,NaN,kg,757.0,NaN,NaN,NaN,NaN,NaN,NaN,C,A,20200101.0,2020.0,52.0,CHE,Switzerland,X,Export,0.0,W00,World,0.0,W00,World,H5,HS,True,Zirconium ores and concentrates,6.0,True,C00,TOTAL CPC,0.0,0.0,TOTAL MOT,8.0,2.500000e+01,True,-1.0,NaN,0.0,False,2.500000e+01,True,0.0,False,0.0,2.665140e+02,2.6

In [23]:
# Join HHI table with (indicators JOIN production)
df_hhi_production_egsehi = pd.merge(df_production_egsehi, df_hhi_master, how="left", left_on=["Year", "Country", "cmdCode"], right_on=["Year", "Country", "cmdCode"])
df_hhi_production_egsehi

,Country,Product,cmdCode,Source_x,Year,Domestic Production Value (P_AC),Value (Ton)_x,qtyUnitAbbr_x,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,no_value_social,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root,Product_x,Source_y,Value,Value (Ton)_y,qtyUnitAbbr_y,reporterCode,reporterDesc_x,Total_value_YearProduct,Share in % (production),Share HHI Production,HHI_production,HHI_production_scaled,typeCode,freqCode,refPeriodId,refYear,refMonth,reporterISO,reporterDesc_y,flowCode,flowDesc,partnerCode,partnerISO,partnerDesc,partner2Code,partner2ISO,partner2Desc,classificationCode,classificationSearchCode,isOriginalClassification,cmdDesc,aggrLevel,isLeaf,customsCode,customsDesc,mosCode,motCode,motDesc,qtyUnitCode,qty,isQtyEstimated,altQtyUnitCode,altQtyUnitAbbr,altQty,isAltQtyEstimated,netWgt,isNetWgtEstimated,grossWgt,isGrossWgtEstimated,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Product_y,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports,HHI_exports_scaled
0,ALBANIA,IRON ORE (FE-CONT.),2601.0,org,1984,4.395020e+08,439502.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALGERIA,IRON ORE (FE-CONT.),2601.0,org,1984,2.000000e+09,2000000.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARGENTINA,IRON ORE (FE-CONT.),2601.0,org,1984,3.455760e+08,345576.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AUSTRALIA,IRON ORE (FE-CONT.),2601.0,org,1984,5.688500e+10,56885000.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AUSTRIA,IRON ORE (FE-CONT.),2601.0,org,1984,1.137872e+09,1137872.0,kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372824,SPAIN,URANIUM (U3O8-CONT.),261210.0,org,2020,NaN,NaN,kg,ESP,0.498795,False,0.646452,False,0.462754,False,0.281208,False,0.373402,False,0.335442,False,0.638047,False,0.005256,2.107548e-14,URANIUM (U3O8-CONT.),org,NaN,NaN,kg,724.0,SPAIN,56269000.0,NaN,NaN,2182.434818,0.182702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
372825,UKRAINE,URANIUM (U3O8-CONT.),261210.0,

In [97]:
# GET entries whose hhi indexes are amepty
# df_hhi_production_egsehi[df_hhi_production_egsehi["HHI_production"].isna() & df_hhi_production_egsehi["HHI_exports"].isna()]

# All the rows have infiormation
# df_hhi_production_egsehi[df_hhi_production_egsehi["HHI_production"].isna()]

# In this case we have a lot of rows with HhI_exports empty because export data contains information from 1970 to 2020 whilst the rest of data is from 2000 to 2020
df_hhi_production_egsehi[df_hhi_production_egsehi["HHI_exports"].isna()  & (df_hhi_production_egsehi["Year"] >= 2000)][["Year", "Country", "Product"]].drop_duplicates().iloc[100:120]

,Year,Country,Product
250,2016,CHINA,ALUMINIUM
251,2017,CHINA,ALUMINIUM
252,2018,CHINA,ALUMINIUM
253,2019,CHINA,ALUMINIUM
254,2020,CHINA,ALUMINIUM
285,2000,CROATIA,ALUMINIUM
286,2001,CROATIA,ALUMINIUM
287,2002,CROATIA,ALUMINIUM
288,2003,CROATIA,ALUMINIUM
289,2004,CROATIA,ALUMINIUM


In [ ]:
df_hhi_master[(df_hhi_master["Year"]==2016) & (df_hhi_master["Country"] == "CHINA")]

In [99]:
df_hhi_master[(df_hhi_master["Year"]==2009) & (df_hhi_master["Country"] == "CROATIA")]

,Year,Country,Value,Product,reporterCode,reporterDesc_x,Total_value_YearProduct,Share in % (production),Share HHI Production,HHI_production,...,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports
294,2009,CROATIA,500.0,ALUMINIUM,191.0,CROATIA,197735840.0,0.000253,6.393950e-08,1713.133234,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6720,2009,CROATIA,NaN,BARYTES,191.0,CROATIA,7662051.0,NaN,NaN,2359.851398,...,NaN,9777.0,9777.0,0.0,True,False,3.581624e+09,0.000142,2.027591e-08,3146.684482
47963,2009,CROATIA,NaN,SILVER,191.0,CROATIA,22322463.0,NaN,NaN,981.801726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# FOR NULL DATA BUT FROM 1970 TO 2020
df_hhi_production_egsehi_1970and2020 = pd.merge(df_hhi_master, df_production_egsehi, how="left", left_on=["Year", "Country", "Product"], right_on=["Year", "Country", "Product"])

In [55]:
df_hhi_production_egsehi_1970and2020

,Year,Country,Value,Product,reporterCode,reporterDesc_x,Total_value_YearProduct,Share in % (production),Share HHI Production,HHI_production,...,value_ecosystems,no_value_ecosystems,value_habitat,no_value_habitat,value_infrastructure,no_value_infrastructure,value_governance_isolated,no_value_governance_isolated,EGSEHI,EGSEHI_6root
0,1970,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,57280000.0,NaN,NaN,1026.755380,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,60982000.0,NaN,NaN,1132.922206,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,68897000.0,NaN,NaN,1128.647391,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,75087000.0,NaN,NaN,1185.642408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,8.0,ALBANIA,83974000.0,NaN,NaN,1223.009237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87512,2020,NaN,NaN,ZIRCONIUM,757.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87513,2020,NaN,NaN,ZIRCONIUM,784.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87514,2020,NaN,NaN,ZIRCONIUM,792.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87515,2020,NaN,NaN,ZIRCONIUM,826.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Save conformed table

In [49]:
df_hhi_production_egsehi

,Year,Country,Domestic Production Value (P_AC),Product,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,...,cifvalue,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports
0,1970,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1971,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1972,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1973,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1974,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69784,2016,RUSSIAN FEDERATION,5.584791e+08,PETROLEUM,RUS,0.624176,False,0.358850,False,0.608937,...,NaN,7.371230e+10,7.371230e+10,0.0,False,True,1.508749e+12,16.892819,285.367322,1154.313467
69785,2017,RUSSIAN FEDERATION,5.585467e+08,PETROLEUM,RUS,0.642995,False,0.365093,False,0.582633,...,NaN,9.337726e+10,9.337726e+10,0.0,False,True,9.716810e+11,NaN,NaN,730.396645
69786,2018,RUSSIAN FEDERATION,5.678849e+08,PETROLEUM,RUS,0.645651,False,0.370905,False,0.607808,...,NaN,1.292011e+11,1.292011e+11,0.0,False,True,1.250927e+12,20.829205,433.855770,978.296637
69787,2019,RUSSIAN FEDERATION,5.733888e+08,PETROLEUM,RUS,0.670757,False,0.378865,False,0.599214,...,NaN,1.222286e+11,1.222286e+11,0.0,False,True,1.261201e+12,21.342775,455.514034,1290.653310


In [57]:
del df_hhi_production_egsehi["cmdCode"]
del df_hhi_production_egsehi_1970and2020["cmdCode"]
# Attach commodity code to each row
classification_codes = {
    "ALUMINIUM": 7616,
    
    "Antimony": 261710,
    "Asbestos": 2524,
    "Barytes": 2511,
    "Bismuth": 8106,
    "Cadmium": 8107,
    "Chromium": 2610,
    "Coal": 2701,
    "Cobalt": 810520,
    "Copper": 2603,
    "Gold": 7108,
    "Graphite": 2504,
    "Iron": 2601,
    "Lead": 2607,
    "Lithium": 283691,
    "Magnesite": 251910,
    "Magnesium": 251910,
    "Manganese": 2602,
    "Mercury": 280540,
    "Molybdenum": 2613,
    "Natural gas": 271111,
    "Nickel": 2604,
    "Petroleum": 2709,
    "Rare earth": 2846,
    "Silver": 261610,
    "Tin": 2609,
    "Titanium": 2614,
    "Tungsten": 2611,
    "Uranium": 261210,
    "Zinc": 2608,
    "Zirconium": 261510,
    "Lithium-ion batteries": 850760 # igual que Litio?
}
classification_codes_df = pd.DataFrame({"Product": [x.upper() for x,y in classification_codes.items()], "cmdCode": [y for x,y in classification_codes.items()]})

df_hhi_production_egsehi = pd.merge(df_hhi_production_egsehi, classification_codes_df, how="left", on=["Product"])
df_hhi_production_egsehi_1970and2020 = pd.merge(df_hhi_production_egsehi_1970and2020, classification_codes_df, how="left", on=["Product"])

df_hhi_production_egsehi


,Year,Country,Domestic Production Value (P_AC),Product,ISO3,value_economic,no_value_economic,value_governance,no_value_governance,value_social,...,fobvalue,primaryValue,legacyEstimationFlag,isReported,isAggregate,Total_netWgt_YearProduct,Share in % (exports),Share HHI Exports,HHI_exports,cmdCode
0,1970,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7616
1,1971,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7616
2,1972,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7616
3,1973,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7616
4,1974,ALBANIA,NaN,ALUMINIUM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69784,2016,RUSSIAN FEDERATION,5.584791e+08,PETROLEUM,RUS,0.624176,False,0.358850,False,0.608937,...,7.371230e+10,7.371230e+10,0.0,False,True,1.508749e+12,16.892819,285.367322,1154.313467,2709
69785,2017,RUSSIAN FEDERATION,5.585467e+08,PETROLEUM,RUS,0.642995,False,0.365093,False,0.582633,...,9.337726e+10,9.337726e+10,0.0,False,True,9.716810e+11,NaN,NaN,730.396645,2709
69786,2018,RUSSIAN FEDERATION,5.678849e+08,PETROLEUM,RUS,0.645651,False,0.370905,False,0.607808,...,1.292011e+11,1.292011e+11,0.0,False,True,1.250927e+12,20.829205,433.855770,978.296637,2709
69787,2019,RUSSIAN FEDERATION,5.733888e+08,PETROLEUM,RUS,0.670757,False,0.378865,False,0.599214,...,1.222286e+11,1.222286e+11,0.0,False,True,1.261201e+12,21.342775,455.514034,1290.653310,2709


In [51]:
df_hhi_production_egsehi.to_csv("../data/hhi_production_exports_egsehi_conformed_table.csv", index=False)

In [164]:
df_hhi_production_egsehi_1970and2020.to_csv("../data/hhi_production_exports_egsehi_null_19702020_conformed_table.csv", index=False)